# SQLAlchemy

The SQLAlchemy models allows for the translation between OO languages like python and relational languages like SQL. It transforms python objects into database information, and the other way around. PSQLAlcheny can connect to many differnt types of sql databases, but all of them follow this standard. The standard specifies the functions that help you connect to a database, send queries, and get results. All the specifications are regulated by PEP 249.

In [1]:
!pip install sqlalchemy

For this topic we will practic on the `Buildings_Database.sqlite`. using the folowing code we can connect to the databse.

In [2]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///Buildings_Database.sqlite', echo=True)
connection = engine.connect()
print(connection, engine)

<sqlalchemy.engine.base.Connection object at 0x0000015990A94460> Engine(sqlite:///Buildings_Database.sqlite)


In [3]:
from sqlalchemy import inspect
inspector = inspect(engine)
inspector.get_table_names()

2023-05-11 18:12:34,243 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2023-05-11 18:12:34,244 INFO sqlalchemy.engine.Engine [raw sql] ()


[]

!! other code in topic https://hyperskill.org/learn/step/12935 that i did not get working

## Data mapping

Mapping transforms SQL objects into Python ones. There are two main mapping types in SQLAlchemy. One is **classical** mapping, the other is **declarative** mapping. We will discuss these types below.

### Classical

Classical mapping refers to the configuration of **a mapped class** that was created with the `mapper()` function. We need to define a database table and the corresponding Python class separately to link them with `mapper()`. After that, all changes to the table and class made via SQLAlchemy are saved in your database. Classical mapping is a **base mapping system** provided by the ORM. Take a look at the snippet below:

````python
from sqlalchemy import Table, MetaData, Column, Integer, String
from sqlalchemy.orm import mapper

metadata = MetaData()

animals = Table('animals', metadata,
                Column('id', Integer, primary_key=True),
                Column('petname', String(30)),
                Column('age', Integer),
                Column('weight', Integer))


class Animals(object):
    def __init__(self, petname, age, weight):
        self.petname = petname
        self.age = age
        self.weight = weight


mapper(Animals, animals)
````

It however does not seem to work with SQLAlchemy 2.0. Must look up right topic

### Declarative

Declarative mapping is a concise version of classical mapping. We don't need to specify a class and a table separately, we can do it all in one class. Let's try to write a concise form of the previous snippet:

!! Had to be changed to match with **SQLAlchemy 2.0**: `from sqlalchemy.orm import declarative_base`

In [4]:
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String

Base = declarative_base()


class Animals(Base):
    __tablename__ = 'animals'

    id = Column(Integer, primary_key=True)
    petname = Column(String(30))
    age = Column(Integer)
    weight = Column(Integer)

In [5]:
Base.metadata.create_all(engine)

2023-05-11 18:12:34,308 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-11 18:12:34,308 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("animals")
2023-05-11 18:12:34,309 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-11 18:12:34,310 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("animals")
2023-05-11 18:12:34,310 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-11 18:12:34,311 INFO sqlalchemy.engine.Engine 
CREATE TABLE animals (
	id INTEGER NOT NULL, 
	petname VARCHAR(30), 
	age INTEGER, 
	weight INTEGER, 
	PRIMARY KEY (id)
)


2023-05-11 18:12:34,311 INFO sqlalchemy.engine.Engine [no key 0.00026s] ()
2023-05-11 18:12:34,314 INFO sqlalchemy.engine.Engine COMMIT


### Session
The `mapper()` function and declarative extensions are primary interfaces for ORM. Once our mappings are configured, we can proceed to the primary interface. It is also known as a session. It helps us to communicate with our database and ensures that all operations run smoothly. You can modify your database and save the changes during the session. To start one, you can use the following statement:

In [6]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()
session

In [7]:
animal_1 = Animals(petname='Timmy', age=2, weight=12)
animal_2 = Animals(petname='Tommy', age=3, weight=10)
animal_3 = Animals(petname='Kitty', age=4, weight=14)
session.add(animal_1)
session.add(animal_2)
session.add(animal_3)
session.commit()

2023-05-11 18:12:34,338 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-11 18:12:34,339 INFO sqlalchemy.engine.Engine INSERT INTO animals (petname, age, weight) VALUES (?, ?, ?)
2023-05-11 18:12:34,340 INFO sqlalchemy.engine.Engine [generated in 0.00066s] ('Timmy', 2, 12)
2023-05-11 18:12:34,341 INFO sqlalchemy.engine.Engine INSERT INTO animals (petname, age, weight) VALUES (?, ?, ?)
2023-05-11 18:12:34,341 INFO sqlalchemy.engine.Engine [cached since 0.001938s ago] ('Tommy', 3, 10)
2023-05-11 18:12:34,342 INFO sqlalchemy.engine.Engine INSERT INTO animals (petname, age, weight) VALUES (?, ?, ?)
2023-05-11 18:12:34,342 INFO sqlalchemy.engine.Engine [cached since 0.002663s ago] ('Kitty', 4, 14)
2023-05-11 18:12:34,343 INFO sqlalchemy.engine.Engine COMMIT


## querying and filtering

 A Query object will generate a SELECT statement for columns of our table and rename each column to a variable name as per PEP convention. Here's what it may look like. Once passed, it will create the animals_petname variable for the table called Animals with the petname column. After this, you'll be able to access the values in the corresponding column via this variable.

In [8]:
from sqlalchemy.orm import Query
query = Query(Animals)
query

In [9]:
print(query)

SELECT animals.id AS animals_id, animals.petname AS animals_petname, animals.age AS animals_age, animals.weight AS animals_weight 
FROM animals


In [10]:
from sqlalchemy.orm import Query, sessionmaker
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:", echo=True)
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

2023-05-11 18:12:34,384 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-11 18:12:34,384 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("animals")
2023-05-11 18:12:34,385 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-11 18:12:34,385 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("animals")
2023-05-11 18:12:34,386 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-11 18:12:34,387 INFO sqlalchemy.engine.Engine 
CREATE TABLE animals (
	id INTEGER NOT NULL, 
	petname VARCHAR(30), 
	age INTEGER, 
	weight INTEGER, 
	PRIMARY KEY (id)
)


2023-05-11 18:12:34,387 INFO sqlalchemy.engine.Engine [no key 0.00048s] ()
2023-05-11 18:12:34,388 INFO sqlalchemy.engine.Engine COMMIT


In [11]:
query = session.query(Animals)
print(query)

SELECT animals.id AS animals_id, animals.petname AS animals_petname, animals.age AS animals_age, animals.weight AS animals_weight 
FROM animals


In [12]:
session.add(Animals(petname="Billy", age=1, weight=8))
session.add(Animals(petname="Susan", age=4, weight=12))

session.commit()

2023-05-11 18:12:34,415 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-11 18:12:34,416 INFO sqlalchemy.engine.Engine INSERT INTO animals (petname, age, weight) VALUES (?, ?, ?)
2023-05-11 18:12:34,416 INFO sqlalchemy.engine.Engine [generated in 0.00034s] ('Billy', 1, 8)
2023-05-11 18:12:34,417 INFO sqlalchemy.engine.Engine INSERT INTO animals (petname, age, weight) VALUES (?, ?, ?)
2023-05-11 18:12:34,418 INFO sqlalchemy.engine.Engine [cached since 0.001548s ago] ('Susan', 4, 12)
2023-05-11 18:12:34,418 INFO sqlalchemy.engine.Engine COMMIT


In [13]:
all_rows = query.all()

2023-05-11 18:12:34,431 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-11 18:12:34,432 INFO sqlalchemy.engine.Engine SELECT animals.id AS animals_id, animals.petname AS animals_petname, animals.age AS animals_age, animals.weight AS animals_weight 
FROM animals
2023-05-11 18:12:34,433 INFO sqlalchemy.engine.Engine [generated in 0.00053s] ()


In [14]:
for row in all_rows:
    print(type(row))  # <class '__main__.Animals'>

<class '__main__.Animals'>
<class '__main__.Animals'>


In [15]:
for row in all_rows:
    print(f"Pet name: {row.petname}, age: {row.age}, weight: {row.weight}")

Pet name: Billy, age: 1, weight: 8
Pet name: Susan, age: 4, weight: 12


In [16]:
query = session.query(Animals.petname)
print(query)

SELECT animals.petname AS animals_petname 
FROM animals


In [17]:
query = session.query(Animals.petname, Animals.age)
for petname, age in query:
    print(petname, age)

2023-05-11 18:12:34,494 INFO sqlalchemy.engine.Engine SELECT animals.petname AS animals_petname, animals.age AS animals_age 
FROM animals
2023-05-11 18:12:34,494 INFO sqlalchemy.engine.Engine [generated in 0.00045s] ()
Billy 1
Susan 4


In [18]:
query = session.query(Animals)
query.count()

2023-05-11 18:12:34,509 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT animals.id AS animals_id, animals.petname AS animals_petname, animals.age AS animals_age, animals.weight AS animals_weight 
FROM animals) AS anon_1
2023-05-11 18:12:34,510 INFO sqlalchemy.engine.Engine [generated in 0.00061s] ()


2

In [19]:
query = session.query(Animals)
for row in query.filter(Animals.petname == "Billy"):
    print(row.petname, row.age, row.weight) # Billy 1 8

2023-05-11 18:12:34,523 INFO sqlalchemy.engine.Engine SELECT animals.id AS animals_id, animals.petname AS animals_petname, animals.age AS animals_age, animals.weight AS animals_weight 
FROM animals 
WHERE animals.petname = ?
2023-05-11 18:12:34,524 INFO sqlalchemy.engine.Engine [generated in 0.00059s] ('Billy',)
Billy 1 8


In [20]:
query = session.query(Animals.age, Animals.weight)

age_gr_than = Animals.age > 2
weight_eq_gr_than = Animals.weight >= 8

for age, weight in query.filter(age_gr_than, weight_eq_gr_than):
    print(f"Pet age: {age}, Pet weight: {weight}")

2023-05-11 18:12:34,540 INFO sqlalchemy.engine.Engine SELECT animals.age AS animals_age, animals.weight AS animals_weight 
FROM animals 
WHERE animals.age > ? AND animals.weight >= ?
2023-05-11 18:12:34,540 INFO sqlalchemy.engine.Engine [generated in 0.00057s] (2, 8)
Pet age: 4, Pet weight: 12


## Updating and deleting

### imports and table creation

In [21]:
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import declarative_base, sessionmaker

Base = declarative_base()

ImportError: cannot import name 'DeclarativeBase' from 'sqlalchemy.orm' (C:\Users\Michiel\anaconda3\lib\site-packages\sqlalchemy\orm\__init__.py)

In [ ]:
class Employee(Base):
    __tablename__ = "employee"

    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    position = Column(String(30))
    salary = Column(Integer)

### Create DB and session

In [ ]:
engine = create_engine("sqlite:///alchemy.db", echo=True)
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

### Fill the session

In [ ]:
session.add(Employee(name="William", position="Data Scientist", salary="12,000"))
session.add(Employee(name="Susan", position="Product Manager", salary="14,000"))
session.add(Employee(name="Andrew", position="Senior Software Engineer", salary="17,000"))
session.add(Employee(name="Nancy", position="Account Executive", salary="13,000"))

session.commit()

### Update methode

In [ ]:
query = session.query(Employee)

query.update({
    "salary": Employee.salary + 1000
})

session.commit()

In [ ]:
empl_filter = query.filter(Employee.name == "William")
empl_filter.update({"salary": Employee.salary + 1000})
session.commit()

In [ ]:
empl_filter = query.filter(Employee.position == "Senior Software Engineer")
empl_filter.update({
    "salary": Employee.salary - 1500,
    "position": "Middle Software Engineer"
})
session.commit()

### Deleting data

In [ ]:
query.filter(Employee.name == "Nancy").delete()
session.commit()

## Migration with Alembic

While working with data, quite often you need to change your database structure: to add a column or a constraint, create, delete, or update a table. The process of changing the schema of a database is called migration. Migrations are something like version control for databases. For sure, you can modify databases using SQL queries, for example. But if you have massive data, it's too much work to do it manually. Moreover, if you make a lot of SQL queries, the code gets more difficult to read. Luckily, there are some migration tools, using which you can make this process much faster, easier, and safer.

**Alembic** is the migration tool that is used with SQLAlchemy. Alembic provides a simple way to create, delete, and change tables and columns in a database. Foremost, you need to install it. You can do this using pip:


In [ ]:
!pip install alembic

After installing it you need to create a migration environment.

In [ ]:
!alembic init alembic

This creates a couple of files:

* `env.py` is a script that is run every time you invoke the alembic migration tool.
* `script.py.maco` is a template file that is used to generate new migration scripts.
* In the `versions` folder, migration scripts will appear.
* The file `alembic.ini` contains all the basic information that makes it possible to connect to the database and is called every time Alembic is used.

Let's create a User model with the columns id, name, surname, and birth in the database users. Pay attention that you also need to configure the database and establish the connection.

In [ ]:
from sqlalchemy import Column, String, Integer, DateTime
from sqlalchemy.orm import declarative_base

Base = declarative_base()
class User(Base):
    __tablename__ = 'user'

    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    surname = Column(String, nullable=False)
    birth = Column(DateTime)

engine = create_engine('sqlite:///users.db')
Base.metadata.create_all(engine)

Now lets see them alembic specific commands".

Add in `alembic.ini`: `sqlalchemy.url = sqlite:///users.db`
Add in `env.py`:
    `from model import Base
    target_metadata = [Base.metadata]`

Next run the following command:

In [ ]:
!alembic revision -m "users"

Apply migration by using:

In [ ]:
!alembic upgrade head

By the way, if you want to roll back to the previous revision, you can run the following command:

In [ ]:
!alembic downgrade -1

## Migration with Flask-Migrate

One more way to make migrations is using Flask-Migrate. Actually, Flask-Migrate is an extension that handles SQLAlchemy database migrations for Flask applications using Alembic. Everything is handled by Alembic, so there is exactly the same functionality.

In [ ]:
!pip install Flask-Migrate

Then, you should create a migration repository with the following command:

In [ ]:
!flask db init

This will create a migrations folder. Let's suppose you have the same model User from the previous section.

Next, let's generate a migration:

In [ ]:
!flask db migrate

And finally, let's apply the migration:

In [ ]:
!flask db upgrade

Great! You are done with it! Then every time you need to change the database models, repeat the migrate and upgrade commands.

While using Flask-Migrate, you can undo the migration as well. Just use the following command:

In [ ]:
!flask db downgrade

## Relationships

Relationship is a connection between tables, one of which has a foreign key that references to the primary key of another table. This allows a database to split and store data in different tables, and at the same time link them together. This is a fundamental feature of relational databases.

There are four basic types of relationships in SQLAlchemy:
* one-to-one, one-to-many
* many-to-one
* many-to-many.

### One-to-one

Suppose, that we have two tables in our database: citizen with columns id, name, and a table passport with columns id, passport_id. For linking them, we will need the relationship() function.

In [21]:
from sqlalchemy import Column, ForeignKey, Integer, Table
from sqlalchemy.orm import declarative_base, relationship

Base = declarative_base()

class Citizen(Base):

    __tablename__ = "citizen"

    id = Column(Integer, primary_key=True)
    name = Column(String(255), unique=True, nullable=False)

    # Relationships
    passport = relationship("Passport")

class Passport(Base):

    __tablename__ = "passport"

    id = Column(Integer, primary_key=True)
    citizen_id = Column(Integer, ForeignKey("citizen.id"))
    passport_id = Column(String(25), unique=True, nullable=False)

    # Relationships
    citizen = relationship("Citizen")

We have linked data between our citizen and passport tables so that fetching one will allow us to get information about the other.

Here, the relationships complement the foreign keys and tell our application (not our database) that we are building a connection between two models. In the example below, the value of the foreign key is `citizen.id`, which is the table name for our `citizen` table. And also we pass the value `Citizen` to our relationship, which is the class name of the data model (not the table name).

### One-to-many

One-to-many relationship connects an object of the first (parent) table with one or more objects of the second (child) table, but not vice versa. Any object of the second table can be associated with only one object of the first table. For example, class-student (in one class, there can be many students, but each student belongs to only one group) or customer-order (a customer can have several orders, but each order can have only one customer). One-to-many relationships are the most common types of database relationships.

In [ ]:
from sqlalchemy.orm import relationship


class User(Base):

    __tablename__ = "user"

    id = Column(Integer, primary_key=True, autoincrement="auto")
    username = Column(String(255), unique=True, nullable=False)
    first_name = Column(String(255))
    last_name = Column(String(255))
    bio = Column(Text)



class Post(Base):


    __tablename__ = "post"

    id = Column(Integer, primary_key=True, index=True)
    user_id = Column(Integer, ForeignKey("user.id"))
    title = Column(String(255), nullable=False)
    body = Column(Text)
    created_at = Column(DateTime, server_default=func.now())
    updated_at = Column(DateTime, server_default=func.now())

    # Relationships
    user = relationship("User")


We have made a connection between user and post tables. For this, we needed to assign a value user.id, which is the name of the user table, and pass the value User to our relationship, the class name of the data model (not the table name).

### Many-to-one
Many-to-many relationship means that any object of the first table can be associated with one or more objects of another table and vice versa. For example, student-discipline (each student can study many disciplines, and each discipline can be taken by many students).

Many-to-many relationships are defined like the one-to-many relationships, but you also need to define an association table and add it to the secondary argument in relationship() in the following way:

In [21]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()


association_table = Table(
    "StudentDicsipline",
    Base.metadata,
    Column("discipline_id", Integer, ForeignKey("discipline.id")),
    Column("student_id", Integer, ForeignKey("student.id")),
)


class Discipline(Base):


    __tablename__ = "discipline"
    id = Column(Integer, primary_key=True)
    name = Column(String)


class Student(Base):


    __tablename__ = "student"
    id = Column(Integer, primary_key=True)
    name = Column(String)
    disciplines = relationship("Discipline", secondary="StudentDiscipline")

In the association table, the names of the columns are defined in the format tablename_columnname. Each column must have a foreign key defined in the format tablename.columnname.